In [1]:
# import dependencies
import os
import pandas as pd
import numpy as np

# read the all_bike_trips.csv file
filepath1 = '../tables/Table7_member_trips.csv'
member_df = pd.read_csv(filepath1, low_memory=False)
filepath2 = '../tables/Table8_casual_trips.csv'
casual_df = pd.read_csv(filepath2, low_memory=False)

In [2]:
# Drop columns to 'startsstationnumber' and 'weekday' only
member_df = member_df[['startsstationnumber','weekday']]
casual_df = casual_df[['startsstationnumber','membertype','weekday']]

In [3]:
# Convert the table into multi-index dataframe with startsstationnumber grouping weekday and count its frequency
member_dayofweek = pd.DataFrame(member_df.groupby('startsstationnumber')['weekday'].value_counts().fillna(0))
member_dayofweek = member_dayofweek.rename(columns = {'weekday':'count'}).reset_index()
member_dayofweek.tail()

,startsstationnumber,weekday,count
4413,32901,Monday,4
4414,32901,Wednesday,4
4415,32901,Friday,2
4416,32902,Wednesday,7
4417,32902,Monday,1


In [4]:
# Convert the table into multi-index dataframe with startsstationnumber grouping weekday and count its frequency
casual_dayofweek = pd.DataFrame(casual_df.groupby('startsstationnumber')['weekday'].value_counts().fillna(0))
casual_dayofweek = casual_dayofweek.rename(columns = {'weekday':'count'}).reset_index()
casual_dayofweek.head(25)

,startsstationnumber,weekday,count
0,0,Saturday,15
1,0,Wednesday,6
2,0,Sunday,4
3,0,Thursday,4
4,0,Monday,2
5,0,Tuesday,2
6,0,Friday,1
7,31000,Saturday,1404
8,31000,Sunday,1246
9,31000,Wednesday,632


In [5]:
# Merge the 'member_dayofweek' and 'casual_dayofweek' together on 'startsstationnumber' and 'weekday'
merge_df = member_dayofweek.merge(casual_dayofweek, 
                                how='outer', 
                                left_on=["startsstationnumber", "weekday"], 
                                right_on=["startsstationnumber", "weekday"]).fillna(0)
merge_df.head(25)

,startsstationnumber,weekday,count_x,count_y
0,0,Thursday,153.0,4.0
1,0,Wednesday,132.0,6.0
2,0,Monday,122.0,2.0
3,0,Tuesday,117.0,2.0
4,0,Friday,101.0,1.0
5,0,Sunday,59.0,4.0
6,0,Saturday,58.0,15.0
7,31000,Wednesday,1883.0,632.0
8,31000,Thursday,1856.0,582.0
9,31000,Tuesday,1794.0,488.0


In [6]:
# Merge the member trip and casual trip dataframes and add ratio column
station_dayofweek_ratio = merge_df.assign(ratio = merge_df['count_x'] / (merge_df['count_x'] + merge_df['count_y'])).round(2)
station_dayofweek_ratio = station_dayofweek_ratio.assign(trip_total = merge_df['count_x']+merge_df['count_y'])
station_dayofweek_ratio = station_dayofweek_ratio[['startsstationnumber','weekday','ratio', 'trip_total',
                                                   'count_x','count_y']].rename(columns = {'count_x':'trip_by_member',
                                                                                           'count_y':'trip_by_casual'
                                                                                           })
station_dayofweek_ratio

,startsstationnumber,weekday,ratio,trip_total,trip_by_member,trip_by_casual
0,0,Thursday,0.97,157.0,153.0,4.0
1,0,Wednesday,0.96,138.0,132.0,6.0
2,0,Monday,0.98,124.0,122.0,2.0
3,0,Tuesday,0.98,119.0,117.0,2.0
4,0,Friday,0.99,102.0,101.0,1.0
...,...,...,...,...,...,...
4457,32094,Wednesday,0.00,28.0,0.0,28.0
4458,32236,Saturday,0.00,53.0,0.0,53.0
4459,32236,Sunday,0.00,31.0,0.0,31.0
4460,32236,Tuesday,0.00,18.0,0.0,18.0


In [7]:
# Reduce columns                                                                                  
station_dayofweek_ratio = station_dayofweek_ratio[['startsstationnumber','weekday','ratio','trip_total']]
station_dayofweek_ratio

,startsstationnumber,weekday,ratio,trip_total
0,0,Thursday,0.97,157.0
1,0,Wednesday,0.96,138.0
2,0,Monday,0.98,124.0
3,0,Tuesday,0.98,119.0
4,0,Friday,0.99,102.0
...,...,...,...,...
4457,32094,Wednesday,0.00,28.0
4458,32236,Saturday,0.00,53.0
4459,32236,Sunday,0.00,31.0
4460,32236,Tuesday,0.00,18.0


In [8]:
# # Output to csv
# folder_path = '../tables/'
# station_dayofweek_ratio.to_csv(os.path.join(folder_path,'Table12_station_dayofweek_ratio_triptotal.csv'),index=False)